In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings

# Ignore warnings for clean output
warnings.filterwarnings('ignore')

# Matplotlib settings
%matplotlib inline
sns.set(style="whitegrid")
